In [14]:
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import json
import pandas as pd
import datetime

In [3]:
# initiating the webdriver. Parameter includes the path of the webdriver.
driver = webdriver.Chrome('./assets/chromedriver') 

In [4]:
def getHTMLContent(URL):
    driver.get(URL)
    
    # this is just to ensure that the page is loaded
    time.sleep(5)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [5]:
def main():
    URL = "https://www.benzinga.com/category/article-type/top-gainers"
    content = getHTMLContent(URL)

In [6]:
URL = "https://www.benzinga.com/category/article-type/top-gainers"
content = getHTMLContent(URL)

In [7]:
def extract_using_regex_safe(pattern, text):
    candidates = re.findall(pattern, text)
    if len(candidates) == 0:
        raise Exception(f"Cannot find text matching pattern: {pattern}")
    elif len(candidates) > 1:
        print(f"WARNING: More than 1 found. Please avoid this function!")
    else:
        return candidates[0]

In [8]:
def get_element_safe(source, tag, class_=None, recursive=True):
    if class_ is None:
        filtered = source.find_all(tag, recursive=recursive)
    else:
        filtered = source.find_all(tag, class_=class_, recursive=recursive)
    if len(filtered) == 0:
        raise Exception(f"Cannot find element: <{tag} class='{class_}'></{tag}>")
    elif len(filtered) > 1:
        print(f"WARNING: More than 1 element found. Please avoid this function!")
    else:
        return filtered[0]

In [9]:
posts_container = get_element_safe(content, 'div', "benzinga-articles benzinga-articles-mixed")

In [10]:
posts = posts_container.find_all('li', recursive=True)

In [16]:
datetime.datetime.now().strftime('%A')

'Wednesday'

In [13]:
for i in range(2):
    URL = f"https://www.benzinga.com/category/article-type/top-gainers?page={i}"
    content = getHTMLContent(URL)
    posts_container = get_element_safe(content, 'div', "benzinga-articles benzinga-articles-mixed")
    posts = posts_container.find_all('li', recursive=True)
    for post in posts:
        try:
            title_container = get_element_safe(post, 'h3')
            title_elem = get_element_safe(title_container, "a")
            title = extract_using_regex_safe("[0-9]{2}.Stocks Moving [^0-9]+$", title_elem.text)
            URL = f"https://www.benzinga.com{title_elem['href']}"
            print(title, URL)
        except:
            continue

34 Stocks Moving In Wednesday's Mid-Day Session https://www.benzinga.com/news/21/08/22560432/34-stocks-moving-in-wednesdays-mid-day-session
22 Stocks Moving in Wednesday's Pre-Market Session https://www.benzinga.com/news/21/08/22552951/22-stocks-moving-in-wednesdays-pre-market-session
48 Stocks Moving In Tuesday's Mid-Day Session https://www.benzinga.com/news/21/08/22539235/48-stocks-moving-in-tuesdays-mid-day-session
25 Stocks Moving in Tuesday's Pre-Market Session https://www.benzinga.com/news/21/08/22530681/25-stocks-moving-in-tuesdays-pre-market-session
38 Stocks Moving In Monday's Mid-Day Session https://www.benzinga.com/news/21/08/22515398/38-stocks-moving-in-mondays-mid-day-session
24 Stocks Moving in Monday's Pre-Market Session https://www.benzinga.com/news/21/08/22507007/24-stocks-moving-in-mondays-pre-market-session


In [19]:
links = []
for i in range(3):
    URL = f"https://www.benzinga.com/category/article-type/top-gainers?page={i}"
    content = getHTMLContent(URL)
    posts_container = get_element_safe(
        content, "div", "benzinga-articles benzinga-articles-mixed"
    )
    posts = posts_container.find_all("li", recursive=True)
    for post in posts:
        try:
            title_container = get_element_safe(post, "h3")
            title_elem = get_element_safe(title_container, "a")
            title = extract_using_regex_safe(
                "[0-9]{2}.Stocks Moving [^0-9]+$", title_elem.text
            )
            URL = f"https://www.benzinga.com{title_elem['href']}"
            links.append([title, URL])
        except Exception:
            continue

current_date = datetime.datetime.now()
weekday_human_text = current_date.strftime("%A")
links = list(
    filter(lambda x: weekday_human_text.lower() not in x[0].lower(), links)
)

links = links[:4]
result = list(map(lambda x: x[1], links))
print(result)

['https://www.benzinga.com/news/21/08/22539235/48-stocks-moving-in-tuesdays-mid-day-session', 'https://www.benzinga.com/news/21/08/22530681/25-stocks-moving-in-tuesdays-pre-market-session', 'https://www.benzinga.com/news/21/08/22515398/38-stocks-moving-in-mondays-mid-day-session', 'https://www.benzinga.com/news/21/08/22507007/24-stocks-moving-in-mondays-pre-market-session']


In [20]:
links = links[:4]
result = list(map(lambda x: x[1], links))
print(result)

['https://www.benzinga.com/news/21/08/22539235/48-stocks-moving-in-tuesdays-mid-day-session', 'https://www.benzinga.com/news/21/08/22530681/25-stocks-moving-in-tuesdays-pre-market-session', 'https://www.benzinga.com/news/21/08/22515398/38-stocks-moving-in-mondays-mid-day-session', 'https://www.benzinga.com/news/21/08/22507007/24-stocks-moving-in-mondays-pre-market-session']


In [93]:
URL = "https://www.benzinga.com/news/21/08/22539235/48-stocks-moving-in-tuesdays-mid-day-session"
content = getHTMLContent(URL)

In [94]:
stocks_container = get_element_safe(content, "div", "article-content-body-only")

In [95]:
stocks = stocks_container.find_all('li', recursive=True)

In [97]:
BULL_VERBS = ["jumped", "climbed", "surged", "gained", "rose"]
BEAR_VERBS = ["dipped", "fell", "dropped", "declined", "tumbled"]
DIRECTION_VERBS = BULL_VERBS + BEAR_VERBS

for stock in stocks:    
    ticker_elem = get_element_safe(stock, tag='a', class_="ticker bztwwidgethover")
    
    ticker = extract_using_regex_safe("[A-Z]{1,5}(?:\s*\d{6}[PC]\d{8})?", ticker_elem.text)
    gap_candidates = extract_using_regex_safe(f"({'|'.join(DIRECTION_VERBS)}) (\d*\.?\d+%?)", stock.text)
    
    isGoingUp = gap_candidates[0] in BULL_VERBS
    gap = float(gap_candidates[1].replace("%", ""))
    
#     print(ticker, isGoingUp, gap)

In [98]:
print(len(stocks))

48


In [99]:
def get_tickers_from_cap_list(cap_name):
    df = pd.read_csv(f'assets/{cap_name.upper()}.csv')
    all_symbols = df['Symbol']
    
    return all_symbols.tolist()

In [100]:
MARKET_CAPS = ["MEGA", "LARGE", "MEDIUM"]
tickers = []
for market_cap in MARKET_CAPS:
    tickers += get_tickers_from_cap_list(market_cap)

In [101]:
len(tickers)

863